In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics, rich
from dataclasses import dataclass, field
from IPython.display import display

In [4]:
@dataclass
class Handshake:
    _df: pd.DataFrame
    start: float
    end: float

    @property
    def df(self):
        return self._df.iloc[self.start:self.end]

    def with_offset(self, offset):
        return Handshake(self._df, self.start-offset, self.end+offset)

    @property
    def start_ms(self):
        return self._df['timestamp'].iloc[self.start]

    @property
    def end_ms(self):
        return self._df['timestamp'].iloc[self.end]

    @property
    def diff(self): # in milliseconds
        return round((self.end_ms - self.start_ms) * 1000, 2)

    @property
    def energy(self): # in millijoules
        time_step = 0.000250 # seconds
        joules = (self.df['power'] * time_step).sum()
        return round(joules * 1000, 2)

    def __str__(self) -> str:
        return f"""Duration (ms): {self.diff}
Energy (mJ): {self.energy}"""

    def __repr__(self) -> str:
        dic = {
            "duration": self.diff,
            "energy": self.energy,
        }
        return str(dic)

class DataLoader:
    csv_files_otii = {
        "current": "Main current - Arc.csv",
        "power": "Main power - Arc.csv",
        "gpio": "GPI 1 - Arc.csv",
    }

    # @lru_cache
    def find_handshakes(df):
        # convert NaN to an arbitrary integer
        df['gpio'] = df['gpio'].fillna(-9).astype(int)

        gpio_values = df['gpio'].values
        in_handshake = False
        hs_start, hs_end = 0, 0
        handshakes = []
        for index, value in enumerate(gpio_values):
            if not in_handshake and value == 1:
                in_handshake = True
                hs_start = index
            elif in_handshake and value == 0:
                in_handshake = False
                hs_end = index
                hs = Handshake(df, hs_start, hs_end)
                handshakes.append(hs)

            if in_handshake:
                gpio_values[index] = 1

        # convert temporary value to 0
        gpio_values[gpio_values == -9] = 0
        # Update the 'gpio' column in the DataFrame
        df['gpio'] = gpio_values
        return df, handshakes

    def run(results_dir):
        dfs = []

        for source, csv_file in DataLoader.csv_files_otii.items():
            filename = f"{results_dir}/{csv_file}"
            df = pd.read_csv(filename)
            df = df.rename(columns={'Timestamp': 'timestamp'})
            df = df.rename(columns={'Value': source})
            # print(df.head())
            dfs.append(df)

        merged_df = dfs[0]
        for df in dfs[1:]:
            merged_df = merged_df.merge(df, on="timestamp", how="outer")

        merged_df, handshakes = DataLoader.find_handshakes(merged_df)

        return merged_df, handshakes

def mean_stdev(arr, ndigits=2):
    return round(statistics.mean(arr), ndigits), round(statistics.stdev(arr), ndigits)

@dataclass
class HandshakeSet:
    label: str
    csv_folder: str
    df: pd.DataFrame = field(default_factory=pd.DataFrame)
    handshakes: list = field(default_factory=list)

    def __post_init__(self):
        self.df, self.handshakes = DataLoader.run(self.csv_folder)

    def __getitem__(self, index):
        return self.handshakes[index]

    def duration_mean_stdev(self):
        return mean_stdev([hs.diff for hs in self.handshakes])

    def energy_mean_stdev(self):
        return mean_stdev([hs.energy for hs in self.handshakes])

    def plot(self, index):
        hs = self[index]
        hs_offset = 200
        xlim_offset = 0.02
        hs_for_plot = hs.with_offset(hs_offset)

        plt.figure(figsize=(18, 2))
        plt.plot(hs_for_plot.df['timestamp'], hs_for_plot.df['power'], label='Power (mW)')

        # draw vertical red lines at start and stop
        plt.xlim(hs.start_ms - xlim_offset, hs.end_ms + xlim_offset)
        plt.axvline(x=hs.start_ms, color='red')
        plt.axvline(x=hs.end_ms, color='red')

        # add duration
        duration_position_x = (hs.start_ms + hs.end_ms) / 2
        y_position = plt.ylim()[1] * 1.03
        plt.text(duration_position_x, y_position, f"{self.label} handshake #{index}\n{hs}", ha='center')

        plt.xlabel('Timestamp (s)')
        plt.ylabel('Power (mW)')

        plt.legend()
        plt.show()

    def __repr__(self) -> str:
        return str(self.summary())

    def summary(self):
        return {
            'label': self.label,
            'handshakes': len(self.handshakes),
            'duration': self.duration_mean_stdev(),
            'energy': self.energy_mean_stdev(),
        }

@dataclass
class HandshakeMemory():
    csv_file: str
    df: pd.DataFrame = None

    def __post_init__(self):
        self.df = pd.read_csv(self.csv_file).set_index('protocol')
        self.df = (self.df / 1000) #.astype(int) # convert to kB

    def plot(self):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        colors = {
            'text': 'tab:blue',
            'data': 'firebrick',
            'bss': 'tab:green', 
            'stack': 'tab:orange'
        }
        xtick_labels = ['EDHOC #20', 'DTLS 1.3']

        self.df[['data', 'bss', 'stack']].plot(kind='bar', stacked=True, ax=ax1, zorder=5, color=[colors['data'], colors['bss'], colors['stack']])
        ax1.set_xticklabels(xtick_labels, rotation=0)  # rotation=0 makes labels horizontal
        ax1.set_title('RAM Usage')
        ax1.set_xlabel('Protocol')
        ax1.set_ylabel('RAM Usage (kB)')

        self.df[['text', 'data']].plot(kind='bar', stacked=True, ax=ax2, zorder=5, color=[colors['text'], colors['data']])
        ax2.set_xticklabels(xtick_labels, rotation=0)
        ax2.set_title('Flash Usage')
        ax2.set_xlabel('Protocol')
        ax2.set_ylabel('Flash Usage (kB)')

        [ax.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, zorder=1) for ax in [ax1, ax2]]

        plt.subplots_adjust(wspace=0.3)
        # plt.tight_layout()
        plt.show()

    def summary(self):
        return {
            'text': self.df['text'].to_dict(),
            'data': self.df['data'].to_dict(),
            'bss': self.df['bss'].to_dict(),
            'stack': self.df['stack'].to_dict(),
        }

@dataclass
class Experiment:
    label: str
    csv_folder_edhoc: str
    csv_folder_dtls: str
    memory_csv_file: str
    edhoc_hs: HandshakeSet = None
    dtls_hs: HandshakeSet = None
    memory: HandshakeMemory = None

    def __post_init__(self):
        self.edhoc_hs = HandshakeSet("edhoc", self.csv_folder_edhoc)
        self.dtls_hs = HandshakeSet("dtls", self.csv_folder_dtls)
        self.memory = HandshakeMemory(self.memory_csv_file)

    def plot_durations(self):
        protocols_names = ['EDHOC #20', 'DTLS 1.3']
        df = self.summary_handshakes_df()

        # Plotting
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        bar_positions = [0, 0.5]

        df['duration'].plot(kind='bar', ax=ax1, yerr=df['duration_std'], capsize=10, zorder=5)
        ax1.set_xticklabels(protocols_names, rotation=0)
        ax1.set_ylabel('Handshake Duration (ms)')
        ax1.set_title('Handshake Duration')

        df['energy'].plot(kind='bar', ax=ax2, yerr=df['energy_std'], capsize=10, zorder=5)
        ax2.set_xticklabels(protocols_names, rotation=0)
        ax2.set_ylabel('Handshake Energy (mJ)')
        ax2.set_title('Handshake Energy Consumption')

        [ax.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, zorder=1) for ax in [ax1, ax2]]

        plt.subplots_adjust(wspace=0.2)
        # plt.tight_layout()
        plt.show()

    def summary_handshakes_df(self):
        edhoc_sum = self.edhoc_hs.summary()
        dtls_sum = self.dtls_hs.summary()
        df = pd.DataFrame({
            'protocol': ['edhoc', 'dtls'],
            'handshakes': [edhoc_sum['handshakes'], dtls_sum['handshakes']],
            'duration': [edhoc_sum['duration'][0], dtls_sum['duration'][0]],
            'duration_std': [edhoc_sum['duration'][1], dtls_sum['duration'][1]],
            'energy': [edhoc_sum['energy'][0], dtls_sum['energy'][0]],
            'energy_std': [edhoc_sum['energy'][1], dtls_sum['energy'][1]]
        })
        df.set_index('protocol', inplace=True)
        return df

    def summary(self):
        return {
            'label': self.label,
            'edhoc_hs': self.edhoc_hs.summary(),
            'dtls_hs': self.dtls_hs.summary(),
            'memory': self.memory.summary(),
        }

    def results(self):
        print("Handshake duration and energy: ")
        display(self.summary_handshakes_df())
        self.plot_durations()
        print("Memory usage: ")
        display(self.memory.df)
        self.memory.plot()
        print("A peek at one particular handshake (red lines mean start/stop of the handshake procedure): ")
        self.edhoc_hs.plot(0)
        self.dtls_hs.plot(0)

# edhoc = HandshakeSet("./results/edhoc-09aug-16h24-csv")
# # edhoc[0].with_offset(2)
# edhoc.summary()

# experiment = Experiment(
#     "edhoc_vs_dtls-09aug",
#     "./results/edhoc-09aug-16h24-csv",
#     "./results/dtls-09aug-17h26-csv",
#     "./results/memory.csv",
# )

# rich.print(experiment.summary())
# experiment.memory.plot()

In [ ]:
experiment = Experiment(
    "edhoc_vs_dtls-09aug",
    "./results/edhoc-21aug-16h22-csv",
    "./results/dtls-21aug-16h26-csv",
    "./results/memory-2023-08-21_15:55:20.csv",
)

experiment.results()